## 發現 Threading 不會比較快，因為整理平行處理完後的資料，運算量很大。所以作罷
## 發現 Swifter 不會比較快，分配平行運算的時間成本很高
## 發現載入資料時間其實不長

Use Nothing more
--- 0.466994047164917 seconds for iter 1 step, each step have 128, so the model train with 128 img this iter ---
--- 4.460038661956787 seconds for iter 10 step, each step have 128, so the model train with 1280 img this iter ---
--- 4.13910174369812 seconds for iter 10 step, each step have 128, so the model train with 1280 img this iter ---


Use Swifter
--- 0.5247406959533691 seconds for iter 1 step, each step have 128, so the model train with 128 img this iter ---
--- 5.089306354522705 seconds for iter 10 step, each step have 128, so the model train with 1280 img this iter ---
--- 4.755419015884399 seconds for iter 10 step, each step have 128, so the model train with 1280 img this iter ---


%matplotlib inline

In [1]:

import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from random import shuffle
import math

import pandas as pd

import pickle

from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing

#from scipy.misc import imread, imresize



In [2]:
train_df = pd.read_csv('../../data/Manually_Annotated_file_lists/training_face_mesh_crop.csv')
train_df_2 = pd.read_csv('../../data/Automatically_annotated_file_list/automatically_annotated_face_mesh_crop.csv')

train_df.subDirectory_filePath = '../../data/Manually_Annotated_Images_FaceMesh_Cropped/' + train_df.subDirectory_filePath
train_df_2.subDirectory_filePath = '../../data/Automatically_Annotated_Images_FaceMesh_Cropped/' + train_df_2.subDirectory_filePath

train_df = train_df.append(train_df_2)
del train_df_2
train_df = train_df[train_df['have_facemesh']]

In [3]:
class DataFrameBatchIterator:
    def __init__(self, dataframe, batch_size):
        self.df = dataframe
        self.index = 0
        self.batch_size = batch_size

    def __iter__(self):
        return self
        
    def __next__(self):
        if self.index < len(train_df):
            self.index += self.batch_size
            return self.df[self.index - self.batch_size: self.index].copy()
        else:
            raise StopIteration

In [4]:
batch_size = 128

In [9]:
import swifter

In [13]:
import threading

In [35]:
func = lambda x: cv2.resize(
         cv2.cvtColor(cv2.imread(x), cv2.COLOR_BGR2RGB), (224, 224))

def process_img(file, all_files, idx):
    all_files[idx] = func(file)



In [36]:
##time
all_files = {}
threads = []
for idx, file in enumerate(list(row.subDirectory_filePath)):
    t = threading.Thread(target = process_img, args = (file, all_files, idx))
    t.start()
    threads.append(t)

for thread in threads:
    thread.join()

In [71]:
imgs = sorted(all_files.items())

In [76]:
shape = imgs[0][1].shape

In [77]:
imgs = (list(map(lambda x: x[1], imgs)))

In [82]:
img_array = np.array([])
for img in (imgs):
    img_array = np.append(img_array, img)

In [81]:
img_array.shape

(19267584,)

In [69]:
type(imgs)

pandas.core.series.Series

In [87]:
import time
start_time = time.time()

total_loss = 0
total_step = 0
for step, row in enumerate(DataFrameBatchIterator(train_df, batch_size=batch_size)):

    # Process Data
#     imgs = row.subDirectory_filePath.apply(lambda x: cv2.resize(
#         cv2.cvtColor(cv2.imread(x), cv2.COLOR_BGR2RGB), (224, 224)))
    imgs = row.subDirectory_filePath.swifter.progress_bar(False).apply(lambda x: cv2.resize(
         cv2.cvtColor(cv2.imread(x), cv2.COLOR_BGR2RGB), (224, 224)))

    total_step += 1

    # Log every 200 batches.
    if step % 10 == 0:
        #print(f"Training loss at step {step}: {total_loss/total_step}")
        #print("Seen so far: %s samples" % ((step + 1) * batch_size))
        print(f"--- {time.time() - start_time} seconds for iter {total_step} step, each step have {batch_size}, so the model train with {total_step*batch_size} img this iter ---")
        total_loss = 0
        total_step = 0
        
        start_time = time.time()

--- 0.5132644176483154 seconds for iter 1 step, each step have 128, so the model train with 128 img this iter ---


KeyboardInterrupt: 

In [94]:
img_array = img_array.reshape(-1, *list(imgs)[0].shape)

y_arousal = np.array(row.arousal)

In [ ]:
total_loss